# 4. Named Entity Recognition

In [1]:
%run __init__.py

In [2]:
import os
import pandas as pd

/home/alejandro/.envs/edma/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [13]:
from collections import Counter

from sklearn.base import TransformerMixin, BaseEstimator
from spacy import displacy


class NamedEntityRecognizer(BaseEstimator, TransformerMixin):
    def __init__(self, spacy_model, disable=None):
        self.nlp = spacy_model.load()
        self.disable = disable if disable is not None else []
    
    def fit(self, X, y=None):
        return self

    def transform(self, X, *args, **kwargs):
        return [self.get_entities(text) for text in X]
    
    def get_entities(self, text):
        doc = self.nlp(text)
        return list(set([(x.text, x.label_) for x in doc.ents 
                if x.label_ not in self.disable
                and len(x.text) > 2]))
    
    def get_most_common_entities(self, text, n=10):
        entities = self.get_entities(text)
        labels = [x[0] for x in entities]
        return Counter(labels).most_common(n)
    
    def visualize_entities(self, text, jupyter=True):
        doc = self.nlp(text)
        displacy.render(doc, jupyter=jupyter, style='ent')


## Agriculture

In [14]:
AGRICULTURE_DATASET_DIR = os.path.join(DATA_DIR, 'agriculture')
PMC_FILE_PATH = os.path.join(AGRICULTURE_DATASET_DIR, 'pmc_dataframe.pkl')

pmc_df = pd.read_pickle(PMC_FILE_PATH)
publications = pmc_df['text_cleaned'].values

In [15]:
import en_core_sci_lg

ner = NamedEntityRecognizer(en_core_sci_lg)

In [16]:
text = publications[-1]

ents = ner.get_entities(text)
ents

[('normality', 'ENTITY'),
 ('denaturation', 'ENTITY'),
 ('July', 'ENTITY'),
 ('II-9', 'ENTITY'),
 ('improve', 'ENTITY'),
 ('Pyronema confluens', 'ENTITY'),
 ('Tilia cordata', 'ENTITY'),
 ('absent', 'ENTITY'),
 ('amplified', 'ENTITY'),
 ('Supplementary', 'ENTITY'),
 ('threshold cycle', 'ENTITY'),
 ('Soil', 'ENTITY'),
 ('increase', 'ENTITY'),
 ('TuGM4119', 'ENTITY'),
 ('nurseries', 'ENTITY'),
 ('management', 'ENTITY'),
 ('decreased', 'ENTITY'),
 ('increased', 'ENTITY'),
 ('ecological requirements', 'ENTITY'),
 ('hydric', 'ENTITY'),
 ('significant', 'ENTITY'),
 ('calcium formate', 'ENTITY'),
 ('year', 'ENTITY'),
 ('Tuber melanosporum', 'ENTITY'),
 ('mixing', 'ENTITY'),
 ('values', 'ENTITY'),
 ('forest functioning', 'ENTITY'),
 ('DNA quantities', 'ENTITY'),
 ('ecosystems', 'ENTITY'),
 ('Burgundy truffle', 'ENTITY'),
 ('literature', 'ENTITY'),
 ('e-value', 'ENTITY'),
 ('dry soil', 'ENTITY'),
 ('species', 'ENTITY'),
 ('Versus T', 'ENTITY'),
 ('Summer', 'ENTITY'),
 ('multifactor analysis of v

In [31]:
ner.get_most_common_entities(text)

[('soil', 60),
 ('aestivum', 37),
 ('trees', 29),
 ('production', 28),
 ('DNA', 26),
 ('temperature', 25),
 ('species', 16),
 ('tree', 15),
 ('II-9', 14),
 ('melanosporum', 12)]

In [10]:
ner.visualize_entities(text[:1500])

In [26]:
import en_core_web_md

disallowed_types = ['CARDINAL', 'DATE', 'MONEY', 'ORDINAL', 'PERCENT', 'QUANTITY', 'TIME']
ner_basic = NamedEntityRecognizer(en_core_web_md, disable=disallowed_types)
ner_basic.visualize_entities(text[:1500])

In [24]:
import en_ner_bc5cdr_md

ner_basic = NamedEntityRecognizer(en_ner_bc5cdr_md)
ner_basic.visualize_entities(text[:1500])

In [25]:
import en_ner_craft_md

ner_basic = NamedEntityRecognizer(en_ner_craft_md)
ner_basic.visualize_entities(text[:1500])

In [ ]:
from src.utils import save_object

output_filename = "ner_system.pkl"
save_object(ner_scispacy, os.path.join(NOTEBOOK_4_RESULTS_DIR, output_filename))